In [1]:
import tweepy
from tweepy import OAuthHandler
import json
import datetime as dt
import os
import time
import sys

In [2]:
def load_api():
    consumer_key = 'SJDjKfNFovZBbL77OaKwKbz9i'
    consumer_secret = 'iNZBaIMcglTCejJEnXwW9FV8QFJ7UP4FluWQsSKzQ7KHAMBYHY'
    access_token = '2821153766-XpXi4TV0ZJMrlpOqkZe7Ynzk9EV5KjbiKig8z6z'
    access_secret = 'zCKqlN2Pm88NYxJxViDU80UrJ4BFMpviaoz9fzsIy5qtX'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    return tweepy.API(auth)

In [3]:
def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
    searched_tweets = []
    while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query + "-filter:retweets", count=remaining_tweets, since_id = str(
                since_id), max_id = str(max_id-1), tweet_mode = 'extended')  #, geocode = geocode)
            print('found', len(new_tweets), 'tweets')
            if not new_tweets:
                print('no tweets found')
                break
                
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
            
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now() + dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break
            
    return searched_tweets, max_id

In [4]:
def get_tweet_id(api, date='', days_ago=10, query='a'):
    if date:
        td = date+dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):', tweet[0].created_at)
        return tweet[0].id

In [5]:
def write_tweets(tweets, filename):
    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')

In [ ]:
def main():
    search_phrases = ["Tokopedia", "TokopediaCare", "#Tokopedia", "#Bukalapak", 
                      "Bukalapak", "BukaBantuan", "ShopeeID", "ShopeeCare", 
                      "#ShopeeID"]
    time_limit = 1.5
    max_tweets = 100
    min_days_old, max_days_old = 0, 1
    location = '-6.2891084,106.7560364,22km'

    for search_phrase in search_phrases:
        print('Search phrase = ', search_phrase)

        name = search_phrase.split()[0]
        json_file_root = name+'/'+name
        os.makedirs(os.path.dirname(json_file_root))
        read_IDs = False

        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ', json_file)
            read_IDs = True

        api = load_api()

        if read_IDs:
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = 1
                # max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        since_id = 0
        # since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) = ', max_id)
        print('since id (starting point) = ', since_id)

        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count = ', count)
            tweets, max_id = tweet_search(
                api, search_phrase, max_tweets, max_id=max_id, since_id=since_id, geocode=location)
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrase[-1]:
                        sys.exit(
                            'Maximum number of empty tweet strings reached - exiting')
                    else:
                        print(
                            'Maximum number of empty tweet strings reached - breaking')
                        break


if __name__ == "__main__":
    main()

('Search phrase = ', 'Tokopedia')
('max id (starting point) = ', -1)
('since id (starting point) = ', 0)
('count = ', 1)
('found', 97, 'tweets')
('found', 3, 'tweets')
('count = ', 2)
('found', 82, 'tweets')
('found', 14, 'tweets')
('found', 1, 'tweets')
('found', 2, 'tweets')
('found', 1, 'tweets')
('count = ', 3)
('found', 80, 'tweets')
('found', 12, 'tweets')
('found', 5, 'tweets')
('found', 3, 'tweets')
('count = ', 4)
('found', 77, 'tweets')
('found', 17, 'tweets')
('found', 4, 'tweets')
('found', 1, 'tweets')
('found', 1, 'tweets')
('count = ', 5)
('found', 73, 'tweets')
('found', 21, 'tweets')
('found', 6, 'tweets')
('count = ', 6)
('found', 98, 'tweets')
('found', 2, 'tweets')
('count = ', 7)
('found', 88, 'tweets')
('found', 11, 'tweets')
('found', 1, 'tweets')
('count = ', 8)
('found', 97, 'tweets')
('found', 2, 'tweets')
('found', 0, 'tweets')
no tweets found
('count = ', 9)
('found', 89, 'tweets')
('found', 11, 'tweets')
('count = ', 10)
('found', 100, 'tweets')
('count = '

('found', 64, 'tweets')
('found', 20, 'tweets')
('found', 5, 'tweets')
('found', 5, 'tweets')
('found', 4, 'tweets')
('found', 0, 'tweets')
no tweets found
('count = ', 2)
('found', 68, 'tweets')
('found', 21, 'tweets')
('found', 10, 'tweets')
('found', 1, 'tweets')
('count = ', 3)
('found', 86, 'tweets')
('found', 0, 'tweets')
no tweets found
('count = ', 4)
('found', 0, 'tweets')
no tweets found
('count = ', 5)
('found', 0, 'tweets')
no tweets found
('count = ', 6)
('found', 0, 'tweets')
no tweets found
Maximum number of empty tweet strings reached - breaking
('Search phrase = ', 'Bukalapak')
('max id (starting point) = ', -1)
('since id (starting point) = ', 0)
('count = ', 1)
('found', 100, 'tweets')
('count = ', 2)
('found', 100, 'tweets')
('count = ', 3)
('found', 100, 'tweets')
('count = ', 4)
('found', 99, 'tweets')
('found', 1, 'tweets')
('count = ', 5)
('found', 97, 'tweets')
('found', 3, 'tweets')
('count = ', 6)
('found', 77, 'tweets')
('found', 23, 'tweets')
('count = ', 7